In [4]:
import pandas as pd
import numpy as np

In [5]:
data=pd.read_csv(r"C:\Users\LENOVO\Downloads\Ineuron\obesedata\notebook\data\ObesityDataSet.csv")
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [6]:
data['NObeyesdad'].unique()

array(['Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II',
       'Obesity_Type_I', 'Insufficient_Weight', 'Obesity_Type_II',
       'Obesity_Type_III'], dtype=object)

In [7]:
wt_map={'Normal_Weight':1, 'Overweight_Level_I':2, 'Overweight_Level_II':3,
       'Obesity_Type_I':4, 'Insufficient_Weight':0, 'Obesity_Type_II':5,
       'Obesity_Type_III':6}
data['NObeyesdad']=data['NObeyesdad'].map(wt_map)
data['NObeyesdad']

0       1
1       1
2       1
3       2
4       3
       ..
2106    6
2107    6
2108    6
2109    6
2110    6
Name: NObeyesdad, Length: 2111, dtype: int64

In [8]:
## Independent and dependent features
X = data.drop(labels=['NObeyesdad'],axis=1)
y = data[['NObeyesdad']]

In [9]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [10]:
categorical_cols

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS'],
      dtype='object')

In [11]:
numerical_cols

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')

In [12]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Onehot Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler(with_mean=False))

    ]

)

# Categorigal Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        # ('ordinalencoder', OrdinalEncoder(categories=[Weather_c,Road_c,order_c,vehicle_c,Festival_c,City_c,])),
       ('onehotencoder', OneHotEncoder()),
        ('scaler', StandardScaler(with_mean=False))
    ] 
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [14]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [15]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out()) ##we use traansform to avoid data leakage.



In [16]:
X_train.head()

,num_pipeline__Age,num_pipeline__Height,num_pipeline__Weight,num_pipeline__FCVC,num_pipeline__NCP,num_pipeline__CH2O,num_pipeline__FAF,num_pipeline__TUE,cat_pipeline__Gender_Female,cat_pipeline__Gender_Male,...,cat_pipeline__SCC_yes,cat_pipeline__CALC_Always,cat_pipeline__CALC_Frequently,cat_pipeline__CALC_Sometimes,cat_pipeline__CALC_no,cat_pipeline__MTRANS_Automobile,cat_pipeline__MTRANS_Bike,cat_pipeline__MTRANS_Motorbike,cat_pipeline__MTRANS_Public_Transportation,cat_pipeline__MTRANS_Walking
0,4.851970,19.262207,3.906979,4.246493,3.841205,2.902776,2.354512,0.683026,0.000000,2.000037,...,0.0,0.0,0.0,2.115312,0.000000,2.427386,0.0,0.0,0.000000,0.0
1,3.176229,18.027517,2.278731,5.608879,3.841205,3.277755,3.531767,0.000000,2.000037,0.000000,...,0.0,0.0,0.0,2.115312,0.000000,0.000000,0.0,0.0,2.295583,0.0
2,5.824233,19.345576,3.846716,4.154837,2.021436,4.575097,2.769013,0.524703,0.000000,2.000037,...,0.0,0.0,0.0,2.115312,0.000000,2.427386,0.0,0.0,0.000000,0.0
3,3.781745,19.075930,4.411788,3.095510,3.841205,3.277755,1.149502,1.139194,0.000000,2.000037,...,0.0,0.0,0.0,2.115312,0.000000,0.000000,0.0,0.0,2.295583,0.0
4,3.336319,17.811618,3.346944,4.594694,1.281183,4.916632,1.602873,1.650810,2.000037,0.000000,...,0.0,0.0,0.0,0.000000,2.169948,0.000000,0.0,0.0,2.295583,0.0


In [17]:
y_train.head()

,NObeyesdad
1211,4
427,1
1429,4
1686,5
1299,4


In [23]:
#Model Training
## Import the necessary libraries:-

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [30]:
def evaluate_model(true, predicted):
    report = classification_report(true, predicted)
    # roc_score = roc_auc_score(true, predicted,multi_class='ovr')
    acc_score = accuracy_score(true, predicted)
    return report,acc_score

## Train multiple models

models = {
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'LogisticRegressionClassifier': LogisticRegression(),
    'SVC': SVC()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    report,acc_score=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("Acc score:",acc_score*100)
    print("Classification Report:",report)

    r2_list.append(acc_score)
    
    print('='*35)
    print('\n')

DecisionTreeClassifier
Model Training Performance
Acc score: 92.42902208201893
Classification Report:               precision    recall  f1-score   support

           0       0.95      0.90      0.92        78
           1       0.80      0.89      0.84        82
           2       0.89      0.89      0.89        88
           3       0.95      0.88      0.91        84
           4       0.95      0.95      0.95       115
           5       0.94      0.98      0.96        96
           6       1.00      0.97      0.98        91

    accuracy                           0.92       634
   macro avg       0.92      0.92      0.92       634
weighted avg       0.93      0.92      0.92       634



LogisticRegressionClassifier
Model Training Performance
Acc score: 84.54258675078864
Classification Report:               precision    recall  f1-score   support

           0       0.85      0.90      0.88        78
           1       0.68      0.66      0.67        82
           2       0.74     